In [4]:
import numpy as np
import pandas as pd
from statsmodels.tsa.arima_model import ARIMA
import statsmodels.api as sm

In [5]:
# Load the historical data
df = pd.read_csv('starbucks_post_process_data.csv')

In [6]:
df.head()

,title,score,post_id,url,comms_num,created,body,author,subreddit,created_date,created_time
0,Why I quit,2020,15y4v3n,https://www.reddit.com/r/starbucks/comments/15...,123,2023-08-22 12:47:54,I requested a day off 3 months in advance so I...,colonade17,starbucks,2023-08-22,12:47:54
1,No one believed me when I said we had a cat in...,1645,1542rhe,https://i.redd.it/w5880yvasycb1.png,189,2023-07-19 18:23:58,"His name is Finnegan, and he's back home now 💖",SuperVanessa007,starbucks,2023-07-19,18:23:58
2,Malicious compliance for a dumb new policy,1726,152oy92,https://i.redd.it/qa6pg67cpncb1.jpg,198,2023-07-18 05:07:43,My NEW DM (how are you gonna just start at my ...,Successful_Climate24,starbucks,2023-07-18,05:07:43
3,I enraged a wild Karen today...,1922,151es25,https://www.reddit.com/r/starbucks/comments/15...,134,2023-07-16 19:15:38,My coworker wears a mask each shift because th...,InvincibleSugar,starbucks,2023-07-16,19:15:38
4,I drew an iced americano in coloured pencil:,1925,14x2lun,https://i.redd.it/brul17ewbebb1.jpg,98,2023-07-11 20:32:09,Hey everyone! I’m an artist and ex-Starbucks b...,JOV_97,starbucks,2023-07-11,20:32:09


### Data Cleaning and Preprocessing

In [7]:
# Assuming 'created' contains Unix timestamps (seconds since epoch)
df['created_date'] = pd.to_datetime(df['created_date'])

### Remove Unnecessary Columns

In [8]:
df = df.drop(['title', 'post_id', 'url', 'comms_num', 'created', 'body','author', 'subreddit', 'created_time'], axis=1)

In [9]:
df.head()

,score,created_date
0,2020,2023-08-22
1,1645,2023-07-19
2,1726,2023-07-18
3,1922,2023-07-16
4,1925,2023-07-11


In [10]:
# Sort the data by the 'created' column and split it into training and testing sets.
df.sort_values(by='created_date', inplace=True, ascending=False)

In [11]:
df.set_index('created_date', inplace=True)

In [12]:
df

,score
created_date,
2023-08-22,2020
2023-07-19,1645
2023-07-18,1726
2023-07-16,1922
2023-07-11,1925
...,...
2018-04-18,1673
2018-04-17,2043
2017-10-08,3798


In [13]:
# Split the data into training and testing sets with test_set containing the newest data
train_set, test_set = df[int(len(df) * 0.2):], df[:int(len(df) * 0.2)]

### Model Training:

In [14]:
# Define the order of the ARIMA model (p, d, q)
p = 1  # Autoregressive order
d = 1  # Differencing order (to make the series stationary)
q = 1  # Moving average order

In [15]:
# Create the ARIMA model using the 'score' column
arima_model = sm.tsa.ARIMA(train_set['score'], order=(p, d, q))

/usr/local/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it is not monotonic and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it is not monotonic and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.11/site-packages/statsm

In [16]:
# Fit the ARIMA model to your training data
arima_model_fit = arima_model.fit()

In [17]:
# Print the summary of the model (optional)
print(arima_model_fit.summary())

                               SARIMAX Results                                
Dep. Variable:                  score   No. Observations:                  798
Model:                 ARIMA(1, 1, 1)   Log Likelihood               -6416.174
Date:                Fri, 06 Oct 2023   AIC                          12838.348
Time:                        11:15:49   BIC                          12852.390
Sample:                             0   HQIC                         12843.743
                                - 798                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1          0.0966      0.022      4.375      0.000       0.053       0.140
ma.L1         -0.9930      0.008   -130.070      0.000      -1.008      -0.978
sigma2       5.72e+05   1.51e+04     37.850      0.0

###  Model Evaluation:

#### Evaluate the performance of your ARIMA model on the testing set. Common evaluation metrics include Mean Absolute Error (MAE), Mean Squared Error (MSE), and Root Mean Squared Error (RMSE).

In [18]:
# Make predictions on the testing data
predictions = arima_model_fit.predict(start=0, end=len(test_set) - 1, typ='levels')

In [19]:
# Calculate evaluation metrics
mae = mean_absolute_error(test_set, predictions)
mse = mean_squared_error(test_set, predictions)
rmse = np.sqrt(mse)

NameError: name 'mean_absolute_error' is not defined

### Forecasting:

In [20]:
# Define the number of periods you want to forecast (e.g., one week)
forecast_periods = 7  # Adjust as needed

In [21]:
# Use the trained ARIMA model to make forecasts
forecast = arima_model_fit.forecast(steps=forecast_periods)

/usr/local/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/usr/local/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(


In [22]:
last_date = df.index[0]

In [23]:
last_date

Timestamp('2023-08-22 00:00:00')

In [24]:
last_date = pd.to_datetime(last_date)  # Assuming 'last_date' is a string

In [25]:
forecast_dates = pd.date_range(start=last_date + pd.DateOffset(1), periods=forecast_periods)

In [26]:
# Create a DataFrame to store the forecasted values and dates
forecast_df = pd.DataFrame({'Date': forecast_dates, 'Forecast': forecast})

In [27]:
# Print the forecasted values for the next week
print(forecast_df)

          Date     Forecast
798 2023-08-23  2581.913387
799 2023-08-24  2160.958289
800 2023-08-25  2120.306838
801 2023-08-26  2116.381145
802 2023-08-27  2116.002043
803 2023-08-28  2115.965433
804 2023-08-29  2115.961898


In [28]:
# Specify the file path where you want to save the model
model_save_path = 'arima_reddit_like_model.pkl'

In [29]:
# Save the ARIMA model to the specified file
arima_model_fit.save(model_save_path)

#### Test ARIMA model

In [39]:
import pickle

In [40]:
with open(model_save_path, 'rb') as model_file:
    arima_model_fit_1 = pickle.load(model_file)

In [48]:
date = '2023-08-22'
new_date = '2023-08-25'

In [53]:
# Assuming you have new_data as your test data
forecast = arima_model_fit_1.forecast(steps=7)

/usr/local/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(


In [54]:
print(forecast)

798    2581.913387
799    2160.958289
800    2120.306838
801    2116.381145
802    2116.002043
803    2115.965433
804    2115.961898
Name: predicted_mean, dtype: float64


In [55]:
forecast_dates = pd.date_range(start=last_date + pd.DateOffset(1), periods=forecast_periods)

# Extract only the date part
forecast_dates = forecast_dates.date

In [65]:
# Assuming forecast_dates is a NumPy array
forecast_dates = pd.to_datetime(forecast_dates)

In [66]:
# Extract only the date part
forecast_dates = forecast_dates.date

In [67]:
# Create a DataFrame to store the forecasted values and dates
forecast_df = pd.DataFrame({'Date': forecast_dates, 'Forecast': forecast})

In [68]:
# Convert the DataFrame to the desired format
data = []
for index, row in forecast_df.iterrows():
    data_point = {"x": str(row['Date']), "y": round(row['Forecast'])}
    data.append(data_point)

forecast_data = [
    {
        "id": "likes",
        "color": "hsl(99, 70%, 50%)",
        "data": data
    }
]

In [69]:
print(forecast_data)

[{'id': 'likes', 'color': 'hsl(99, 70%, 50%)', 'data': [{'x': '2023-08-23', 'y': 2582}, {'x': '2023-08-24', 'y': 2161}, {'x': '2023-08-25', 'y': 2120}, {'x': '2023-08-26', 'y': 2116}, {'x': '2023-08-27', 'y': 2116}, {'x': '2023-08-28', 'y': 2116}, {'x': '2023-08-29', 'y': 2116}]}]
